In [84]:
# this is a proof of concept on how we intend of 
# measuring similarity between people in the community 

import torch
from transformers import BertTokenizer, BertModel
from scipy.spatial.distance import cosine

In [85]:

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

user1 = """my name is Hachem Betrouni, I am a Data Science and AI student from Algeria,
            currently working at InstaDeep on predicting and generating new variants of Sars-CoV-2 using GANs"""

user2 = """mohcen is an natural language processing expert from algeria, worked with instadeep 
            on predicting biological properties of protein sequences"""

user3 = """talks about innovation and startups, passionate about the food industry"""

users = [user1, user2, user3]
users = ["[CLS] " + user + " [SEP]" for user in users]

tokenized_users = [tokenizer.tokenize(user) for user in users]


indexed_tokens = [tokenizer.convert_tokens_to_ids(tokenized_user) for tokenized_user in tokenized_users]
segments_ids = [[1] * len(tokenized_user) for i, tokenized_user in zip(range(1,len(users)+1),tokenized_users)]

tokens_tensor = [torch.tensor([indexed_token]) for indexed_token in indexed_tokens]
segments_tensors = [torch.tensor([segments_id]) for segments_id in segments_ids]


In [ ]:

# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

In [87]:
# Run the text through BERT, and collect all of the hidden states produced
# from all 12 layers. 
with torch.no_grad():
    hidden_states = []
    for tokens, segments in zip(tokens_tensor, segments_tensors):
        
        outputs = model(tokens, segments)
        hidden_states.append(outputs[2])

In [88]:
token_embeddings = [torch.stack(h, dim=0) for h in hidden_states]
token_embeddings = [torch.squeeze(t_e, dim=1) for t_e in token_embeddings]
token_embeddings = [t_e.permute(1,0,2) for t_e in token_embeddings]

In [89]:
# `token_vecs` is a tensor with shape [22 x 768]
token_vecs = [h[-2][0] for h in hidden_states]

In [90]:

# Calculate the average of all 22 token vectors.
sentence_embedding = [torch.mean(token_vec, dim=0) for token_vec in token_vecs]

In [96]:
#measuring cosine similarity 

#user 1 and user 2
print("similarity between user1 and user2 :", 1 - cosine(sentence_embedding[0], sentence_embedding[1]))
#user 2 and user 3
print("similarity between user1 and user3 :", 1 - cosine(sentence_embedding[0], sentence_embedding[2]))

similarity between user1 and user2 : 0.9124594330787659
similarity between user1 and user3 : 0.7569449543952942
